In [1]:
import csv
import requests

from subprocess import Popen, PIPE, STDOUT
from pprint import pprint

from utils.file_utils import load_json, dump_json
from default import GOOD_ANCHORS_FILE, POPULATION_CITY_FILE, CITIES_500_FILE

## City coordinates

In [ ]:
good_anchors = load_json(GOOD_ANCHORS_FILE)

In [ ]:
anchors_with_location = []

for anchor in good_anchors:
    ip = anchor['address_v4']
    lat = anchor['geometry']['coordinates'][1]
    lon = anchor['geometry']['coordinates'][0]
    country_code = anchor['country_code']
    anchors_with_location.append({'target_ip': ip, 'lat': lat, 'lon': lon,
                'country_code': country_code})


for anchor in anchors_with_location:
    if 'city' in anchor:
        anchors_with_location.append(anchor)
    else:
        url = f"http://nominatim.openstreetmap.org/reverse?format=geojson&lat={anchor['lat']}&lon={anchor['lon']}"
        r = requests.get(url)
        elem = r.json()
        if 'features' not in elem or len(elem['features']) != 1:
            continue
        info = elem['features'][0]
        if 'properties' not in info or 'address' not in info['properties']:
            continue
        address = info['properties']['address']
        if 'city' in address:
            anchor['city'] = address['city']
        elif 'village' in address:
            anchor['city'] = address['village']
        elif 'town' in address:
            anchor['city'] = address['town']
        elif 'country' in address:
            anchor['city'] = address['country']
        else:
            pprint(info)
            break
        anchors_with_location.append(anchor)


dump_json(anchors_with_location, POPULATION_CITY_FILE)

## test

In [ ]:
anchors_with_pop = load_json(POPULATION_CITY_FILE)

max_pop = 0
max_v = None
for p in anchors_with_pop:
    if p['density'] > max_pop:
        max_pop = p['density']
        max_v = p

pprint(max_v)

## City population

In [ ]:
anchors_with_pop = load_json(POPULATION_CITY_FILE)

In [ ]:
population_threshold = 100000
population_by_city = {}

with open(CITIES_500_FILE) as csvfile:
    reader = csv.reader(csvfile, delimiter="\t", quotechar='"', )
    next(reader, None)
    for row in reader:
        city = row[1]
        ascii_city = row[2]
        # Iso2 code
        country = row[8]
        population = row[14]
        city_key = f"{city}_{country}"
        ascii_city_key = f"{ascii_city}_{country}"
        if population != "":
            population = int(float(population))
            if population >= population_threshold:
                population_by_city[ascii_city_key] = population
                population_by_city[city_key] = population


print(len(population_by_city))

In [ ]:
for anchor in anchors_with_pop:
    try:
        anchor['population'] = population_by_city[f"{anchor['city']}_{anchor['country_code']}"]
    except:
        try:
            cmd = f"cat {CITIES_500_FILE} | grep \"{anchor['city']}\""
            print(cmd)
            process = Popen(cmd, stdout=PIPE, stderr=STDOUT, shell=True)
            output, err = process.communicate()
            row = output.decode().split("\t")
            anchor['population'] = int(row[14])
            anchors_with_pop.append(anchor)
        except Exception as e:
            pprint(e)
            pprint(row)
            pprint(anchor)
            print(f"{anchor['lat']}, {anchor['lon']}")
            anchor['population'] = 0
            anchors_with_pop.append(anchor)


dump_json(anchors_with_pop, POPULATION_CITY_FILE)